In [51]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [52]:
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic

In [53]:
import numpy as np
import matplotlib.dates as mdates

In [54]:
import os
import json
import numpy as np

In [55]:
df = pd.read_json("newsdata.json", lines=True)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   link               209527 non-null  object        
 1   headline           209527 non-null  object        
 2   category           209527 non-null  object        
 3   short_description  209527 non-null  object        
 4   authors            209527 non-null  object        
 5   date               209527 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.6+ MB


In [57]:
df['text'] = (
    df['headline'].fillna('') + ". " +
    df['short_description'].fillna('')
).str.strip()

In [58]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

df['time_bin'] = df['date'].dt.to_period('M').dt.to_timestamp()

df.head()

,link,headline,category,short_description,authors,date,text,time_bin
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,Over 4 Million Americans Roll Up Sleeves For O...,2022-09-01
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,"American Airlines Flyer Charged, Banned For Li...",2022-09-01
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,23 Of The Funniest Tweets About Cats And Dogs ...,2022-09-01
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,The Funniest Tweets From Parents This Week (Se...,2022-09-01
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,Woman Who Called Cops On Black Bird-Watcher Lo...,2022-09-01


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   link               209527 non-null  object        
 1   headline           209527 non-null  object        
 2   category           209527 non-null  object        
 3   short_description  209527 non-null  object        
 4   authors            209527 non-null  object        
 5   date               209527 non-null  datetime64[ns]
 6   text               209527 non-null  object        
 7   time_bin           209527 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(6)
memory usage: 12.8+ MB


In [60]:
df = df[df['text'].str.len() > 10].reset_index(drop=True) # filtering short texts in the dataset

In [61]:
df = df.head(75000) #limitng the data for faster model fitting and output generation


In [62]:
df

,link,headline,category,short_description,authors,date,text,time_bin
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,Over 4 Million Americans Roll Up Sleeves For O...,2022-09-01
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,"American Airlines Flyer Charged, Banned For Li...",2022-09-01
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,23 Of The Funniest Tweets About Cats And Dogs ...,2022-09-01
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,The Funniest Tweets From Parents This Week (Se...,2022-09-01
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,Woman Who Called Cops On Black Bird-Watcher Lo...,2022-09-01
...,...,...,...,...,...,...,...,...
74995,https://www.huffingtonpost.com/entry/pope-fran...,"At Mexico-U.S. Border, Pope Francis Decries Su...",RELIGION,"CIUDAD JUAREZ, Mexico (Reuters) - Pope Francis...","Philip Pullella and Gabriel Stargardter, Reuters",2016-02-18,"At Mexico-U.S. Border, Pope Francis Decries Su...",2016-02-01
74996,https://www.huffingtonpost.com/entry/apple-app...,Apple To Fight Order To Break Encryption On Sa...,TECH,"""We can find no precedent for an American comp...",,2016-02-17,Apple To Fight Order To Break Encryption On Sa...,2016-02-01
74997,https://www.huffingtonpost.com/entry/mattel-th...,This $300 3-D Printer Makes Toy Design Child’s...,GOOD NEWS,ThingMaker will allow kids to make and print t...,Elyse Wanshel,2016-02-17,This $300 3-D Printer Makes Toy Design Child’s...,2016-02-01
74998,https://www.huffingtonpost.comhttp://www.balti...,"Barksdale, Inspiration Behind Characters On 'T...",ENTERTAINMENT,"Nathan Barksdale, the former Baltimore gangste...",,2016-02-17,"Barksdale, Inspiration Behind Characters On 'T...",2016-02-01


In [63]:
embed_model = SentenceTransformer("all-MiniLM-L6-v2")


In [64]:
topic_model = BERTopic(embedding_model=embed_model, verbose=True)

texts = df['text'].tolist()
topics, probs = topic_model.fit_transform(texts)

2025-12-07 20:49:34,558 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 2344/2344 [06:53<00:00,  5.67it/s]
2025-12-07 20:56:31,407 - BERTopic - Embedding - Completed ✓
2025-12-07 20:56:31,409 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-07 20:57:35,970 - BERTopic - Dimensionality - Completed ✓
2025-12-07 20:57:35,976 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-07 20:57:51,293 - BERTopic - Cluster - Completed ✓
2025-12-07 20:57:51,330 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-07 20:57:55,323 - BERTopic - Representation - Completed ✓


In [65]:
df['topic'] = topics
df['topic_prob'] = probs

In [66]:
topic_info = topic_model.get_topic_info()
valid_topics = topic_info[topic_info["Topic"] >= 0]["Topic"].tolist() #removing outlier topic

In [67]:
topic_names = {}

for t in valid_topics:
    words = topic_model.get_topic(t)
    topic_names[t] = ", ".join([w[0] for w in words[:4]])  # top 5 words



In [68]:
df["topic_name"] = df["topic"].map(topic_names)

df = df[df["topic"] != -1].reset_index(drop=True) #removing


In [69]:
topics_over_time = (
    df.groupby(['time_bin', 'topic'])
      .size()
      .reset_index(name='Frequency')
)


In [70]:
pivot = topics_over_time.pivot_table(
    index='time_bin',
    columns='topic',
    values='Frequency',
    fill_value=0
).sort_index()




In [71]:
pivot = pivot.drop(columns=[-1], errors='ignore')

pivot_norm = pivot.div(pivot.sum(axis=1), axis=0).fillna(0)

In [72]:
rolling_mean = pivot_norm.rolling(3, min_periods=1, center=False).mean()

rolling_std = pivot_norm.rolling(3, min_periods=1, center=False).std(ddof=0)

std_floor = 1e-6
rolling_std = rolling_std.clip(lower=std_floor)

zscore = (pivot_norm - rolling_mean) / rolling_std
zscore = zscore.replace([np.inf, -np.inf], np.nan).fillna(0)


In [73]:
#zscore = (pivot_norm - rolling_mean) / rolling_std
#zscore = zscore.fillna(0)

In [74]:

topk = 6

top_k_ids = pivot_norm.sum().sort_values(ascending=False).head(topk).index.tolist()

top_topics = [topic_names[t] for t in top_k_ids]



In [75]:
os.makedirs("saved_trains", exist_ok=True)

embeddings = embed_model.encode(texts, show_progress_bar=True, batch_size=64)
embeddings = embeddings.astype(np.float32)   
np.save("saved_trains/embeddings.npy", embeddings)

topic_model.save("saved_trains/bertopic_model")

with open("saved_trains/topic_names.json", "w", encoding="utf-8") as f:
    json.dump(topic_names, f, ensure_ascii=False, indent=2)

metadata = {
    "embedding_model": "all-MiniLM-L6-v2",
    "n_documents": len(texts),
    "bertopic_version": topic_model.__version__ if hasattr(topic_model, "__version__") else None,
    "saved_at": pd.Timestamp.now().isoformat()
}
with open("saved_trains/metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print("Saved: saved_trains/embeddings.npy, saved_trains/bertopic_model/, saved_trains/topic_names.json, saved_trains/metadata.json")


Batches: 100%|██████████| 1172/1172 [06:40<00:00,  2.92it/s]
2025-12-07 21:04:43,791 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


Saved: saved_trains/embeddings.npy, saved_trains/bertopic_model/, saved_trains/topic_names.json, saved_trains/metadata.json
